まずは、Generative AI Hub SDK の公式ドキュメント[https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/README_sphynx.html]に従って操作する。

In [ ]:
# 環境変数の読み込み
from dotenv import load_dotenv
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI

load_dotenv(verbose=True)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "Hello, World!"),
]

chat_llm = ChatOpenAI(deployment_id='deb90f258a38f739')
chat_llm.invoke(messages)

In [ ]:
# ▶ Notebook Cell 2
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(deployment_id='d6835c4c6c0da6e6')

single_vector = embedding_model.embed_query("Hello world")
print(str(single_vector)[:100])

In [ ]:
# ▶ Notebook Cell 3
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper(k=5) # return top‑5 results

google_tool = Tool.from_function(
    name="google_search",
    description="Search Google and return the first results",
    func=search.run
)


In [ ]:
# ▶ Notebook Cell 4
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=[google_tool],
    llm=chat_llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke("Who won the Tokyo Marathon in 2025, and what was the finishing time?")

In [ ]:
# ▶ Notebook Cell 5
import random

def omikuji(_: str) -> str:
    return random.choice([
        "XXXXXX"
    ])

omikuji_tool = Tool.from_function(
    name="xxxxxx",
    description="......",
    # ?????
)

omikuji_agent = initialize_agent(
    tools=XXXXXXXX,
    llm=chat_llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

omikuji_agent.invoke("おみくじを引いてください")

In [ ]:
# ▶ Notebook Cell 6
import os

from dotenv import load_dotenv
from hdbcli import dbapi

load_dotenv()
# Use connection settings from the environment
connection = dbapi.connect(
    address=os.environ.get("HANA_DB_ADDRESS"),
    port=os.environ.get("HANA_DB_PORT"),
    user=os.environ.get("HANA_DB_USER"),
    password=os.environ.get("HANA_DB_PASSWORD"),
    autocommit=True,
    sslValidateCertificate=False,
)

In [9]:
!pip install langchain-hana

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 27.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.26.4-cp313-cp313-macosx_15_0_arm64.whl size=4728956 sha256=94d8bdd25ebf46256c9312489157522ec03edb151ff9ff7087a7d58f06fbf7ef
  Stored in directory: /Users/I742374/Library/Caches/pip/wheels/8b/2d/9f/b6b46373f328e2ef50388915d351ccacbedac929459b5459bf
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [10]:
from langchain_hana import HanaInternalEmbeddings

embeddings = HanaInternalEmbeddings(internal_embedding_model_id="SAP_NEB.20240715")

In [11]:
from langchain_hana import HanaDB

db = HanaDB(
    embedding=embeddings, connection=connection, table_name="STATE_OF_THE_UNION"
)

In [25]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import TextSplitter

text_documents = TextLoader(
    "state_of_the_union.txt"
).load()
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = ". ",
    chunk_size = 1000,
    chunk_overlap = 0,  # チャンクオーバーラップの文字数
)
text_chunks = text_splitter.split_documents(text_documents)
print(f"Number of document chunks: {len(text_chunks)}")

Number of document chunks: 286


In [27]:
# Delete already existing documents from the table
db.delete(filter={})

# add the loaded document chunks
db.add_documents(text_chunks)

[]

In [28]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query, k=2)

for doc in docs:
    print("-" * 80)
    print(doc.page_content)

--------------------------------------------------------------------------------
We pledge historic resources toward education, to ensure the promise of America reaches every doorstep. In the face of unprecedented challenges, we boldly pursue global leadership. Our administration will champion democracy, so that tomorrow's breakthroughs are born on our shores. My fellow citizens, it is with determination that we strive for prosperity. Across every state, communities are already embracing democracy, to ensure the promise of America reaches every doorstep. This year, united as one people, we set our sights on security. Across every state, communities are already embracing energy, to secure a future worthy of our proud history. This year, united as one people, we set our sights on science. We pledge historic resources toward opportunity, to secure a future worthy of our proud history. This year, united as one people, we set our sights on freedom. Across every state, communities are alread